In [1]:
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import ast
from collections import Counter
import re
import demoji
import requests

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Loading Dataset File

In [3]:
df = pd.read_excel('data/04-VERA-ARB_Raw-dataset.xlsx')

In [4]:
df.head()

,tweet_id,possibly_sensitive,retweet_count,reply_count,like_count,quote_count,bookmark_count,impression_count,user_id,lang,...,protected,followers_count,following_count,tweet_count,listed_count,user_like_count,user_url,verified,user_copyright,user_withheld_country_codes
0,1697950090223026335,True,1,2,5,0,1,2105,1688239842239385600,ar,...,False,5,26,181,0,140,https://www.facebook.com/profile.php?id=100087982050310&mibextid=ZbWKwL,False,NaN,[]
1,1697935446666510767,False,14,21,342,1,3,9203,492235460,ar,...,False,588292,486,43946,121,44406,NaN,False,NaN,[]
2,1620816468328972288,False,32,148,390,4,2,30203,492235460,ar,...,False,588292,486,43946,121,44406,NaN,False,NaN,[]
3,1575998679198367744,False,2,3,249,2,0,0,492235460,ar,...,False,588292,486,43946,121,44406,NaN,False,NaN,[]
4,1697673466311262549,False,2,46,151,3,12,165156,1566066774583435265,ar,...,False,89128,1138,60603,275,8069,NaN,False,NaN,[]


In [5]:
df.shape

(20084, 54)

In [6]:
df.columns

Index(['tweet_id', 'possibly_sensitive', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'bookmark_count', 'impression_count',
       'user_id', 'lang', 'annotations_text', 'annotations_type', 'cashtags',
       'hashtags', 'mentions', 'urls', 'urls_titles', 'urls_desc',
       'context_domain', 'context_domain_desc', 'context_entity',
       'context_entity_desc', 'copyright', 'withheld_country_codes',
       'created_at', 'edits_remaining', 'is_edit_eligible', 'conversation_id',
       'text', 'reply_settings', 'claim', 'label', 'binary_label',
       'user_created_at', 'user_description', 'user_description_urls',
       'user_description_hashtags', 'user_description_mentions',
       'user_description_cashtags', 'location', 'display_name', 'user_name',
       'pinned_tweet_id', 'profile_image_url', 'protected', 'followers_count',
       'following_count', 'tweet_count', 'listed_count', 'user_like_count',
       'user_url', 'verified', 'user_copyright',
       'us

# Dataset Cleaning

In [7]:
df.isnull().sum()

tweet_id                           0
possibly_sensitive                 0
retweet_count                      0
reply_count                        0
like_count                         0
quote_count                        0
bookmark_count                     0
impression_count                   0
user_id                            0
lang                               0
annotations_text                   0
annotations_type                   0
cashtags                           0
hashtags                           0
mentions                           0
urls                               0
urls_titles                        0
urls_desc                          0
context_domain                     0
context_domain_desc                0
context_entity                     0
context_entity_desc                0
copyright                      20021
withheld_country_codes             0
created_at                         0
edits_remaining                    0
is_edit_eligible                   0
c

## Reset claim ids to start with 1 and to be in sequence

In [8]:
df['claim_id'] = df['claim'].rank(method='dense').astype(int) - 1
df['claim_id'] += 1

In [9]:
df[['claim_id', 'claim']].head(10)

,claim_id,claim
0,1,0
1,2,1
2,517,595
3,758,975
4,3,2
5,323,362
6,599,718
7,625,755
8,631,763
9,673,839


In [10]:
df.drop(['cashtags', 'conversation_id', 'claim', 'user_description_cashtags', 'display_name', 'user_name',
         'pinned_tweet_id', 'profile_image_url', 'user_url', 'user_copyright', 'user_withheld_country_codes',
         'label'], axis=1, inplace=True)

In [11]:
df = df.rename(columns={'binary_label': 'label'})

In [12]:
df['copyright'] = df['copyright'].astype(bool)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20084 entries, 0 to 20083
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   tweet_id                   20084 non-null  int64 
 1   possibly_sensitive         20084 non-null  bool  
 2   retweet_count              20084 non-null  int64 
 3   reply_count                20084 non-null  int64 
 4   like_count                 20084 non-null  int64 
 5   quote_count                20084 non-null  int64 
 6   bookmark_count             20084 non-null  int64 
 7   impression_count           20084 non-null  int64 
 8   user_id                    20084 non-null  int64 
 9   lang                       20084 non-null  object
 10  annotations_text           20084 non-null  object
 11  annotations_type           20084 non-null  object
 12  hashtags                   20084 non-null  object
 13  mentions                   20084 non-null  object
 14  urls  

# 1- Textual Features

## Text cleaning

In [14]:
# Function to filter non-Arabic letters and emojis from a string
def filter_text(text):
    try:
        # Remove URLs starting with 'http'
        text = re.sub(r'http\S+', '', text)
        # Remove hashtags starting with '#'
        #text = re.sub(r'#\w+', '', text)
        text = text.replace('_', ' ').replace('\n', ' ')
        arabic_pattern = '[\u0600-\u06FF0-9\s\:\.]+'
        arabic_text = re.findall(arabic_pattern, text)
        return ''.join(arabic_text)
    except:
        return ''

In [15]:
filter_text('بعد إعلان امريكا إغتياله ،،\n\nأيمن الظواهري يظهر في إصدار مرئي لتنظيم القاعدة اليوم بعنوان " دعوة الإمام إلى الامة الإسلامية". https://t.co/ejSBFOFURY')

'بعد إعلان امريكا إغتياله ،،  أيمن الظواهري يظهر في إصدار مرئي لتنظيم القاعدة اليوم بعنوان  دعوة الإمام إلى الامة الإسلامية. '

In [16]:
# Create a new column 'filtered_text' with filtered text
df['text_arabic'] = df['text'].apply(filter_text)

In [17]:
df[['text', 'text_arabic']].sample(10)

,text,text_arabic
4619,قناص فرنسي ع السطح 🤔أنها حرب اهليه هكذا ف فرنسا 🔥 https://t.co/IkfGIoFNd4,قناص فرنسي ع السطح أنها حرب اهليه هكذا ف فرنسا
503,#مصر ترسل معدات وطواقم هندسية عبر معبر رفح البري، لإزالة ركام المنازل المهدمة في #غزة نتيجة القصف الإسرائيلي، تمهيداً لبدء عملية إعمار القطاع. https://t.co/K9uBNyPHap,مصر ترسل معدات وطواقم هندسية عبر معبر رفح البري، لإزالة ركام المنازل المهدمة في غزة نتيجة القصف الإسرائيلي، تمهيداً لبدء عملية إعمار القطاع.
5835,الف الحمد لله والشكر لله .. طائرة حجاج اندونيسيه تسقط اليوم في ايران وكافة ركابها بخير https://t.co/754XimukUh,الف الحمد لله والشكر لله .. طائرة حجاج اندونيسيه تسقط اليوم في ايران وكافة ركابها بخير
3568,رسميا .. #الاهلي السعودي يتعاقد مع الجزائي رياض محرز حتى 2027 مقابل 30 مليون باوند https://t.co/ev7Fq6bNyQ,رسميا .. الاهلي السعودي يتعاقد مع الجزائي رياض محرز حتى 2027 مقابل 30 مليون باوند
11859,🚨 M.E.N \n\nتين هاغ قرر استبعاد كريستيانو رونالدو من قائمة الفريق لمباراة مانشستر يونايتد ضد تشيلسي. https://t.co/pDxI6L83OP,.. تين هاغ قرر استبعاد كريستيانو رونالدو من قائمة الفريق لمباراة مانشستر يونايتد ضد تشيلسي.
9826,"#عاجل | \n\nقيادة القوات المشتركة للتحالف ""تحالف دعم الشرعية في اليمن"":\n\n▪ القوات الخاصة السعودية تلقي القبض على أمير تنظيم داعش الإرهابي باليمن، الملقب بـ ( أبو أسامة المهاجر ) والمسؤول المالي للتنظيم وعدد من أعضاء التنظيم المرافقين له. https://t.co/u6zVBIQt3L",عاجل قيادة القوات المشتركة للتحالف تحالف دعم الشرعية في اليمن: القوات الخاصة السعودية تلقي القبض على أمير تنظيم داعش الإرهابي باليمن، الملقب بـ أبو أسامة المهاجر والمسؤول المالي للتنظيم وعدد من أعضاء التنظيم المرافقين له.
1815,مو صلاح باقي ويتمدد ❤️‍🔥👑 https://t.co/NMtbDu6GgC,مو صلاح باقي ويتمدد
18886,🔻 عاجل\n\nضبط أكبر شحنة مخـــ،،ــــدرات على مستوى الشرق الاوسط في محافظة الانبار. ليش ايraن تدزين مخدرات للانبار لانك تحدين الانبار من الغرب\n\nhttps://t.co/MlWPpLPp06 https://t.co/1qK009fdQe,عاجل ضبط أكبر شحنة مخـــ،،ــــدرات على مستوى الشرق الاوسط في محافظة الانبار. ليش اين تدزين مخدرات للانبار لانك تحدين الانبار من الغرب
14934,صفعة من أمير قطر إلى منتخب_الشواذ أمير قطر الشيخ تميم بن حمد واللاعب التركي مسعود أوزيل يُرسلوا لكم السلام 🇶🇦😉🇹🇷 https://t.co/EH5NgZOjFI,صفعة من أمير قطر إلى منتخب الشواذ أمير قطر الشيخ تميم بن حمد واللاعب التركي مسعود أوزيل يُرسلوا لكم السلام
7861,من ٥ ايام حصلت جريمة في صربيا \nطفل عمره ١٤ سنة كان بيتعرض للتنمر من ٧ بنات و ولد زمايله في المدرسة وقعد يهددهم بالقتل \nزمايله دول زادوا من التنمر وبقوا يعملوا صوت كده ( بيو بيو ) تريقة يعني انو هيضربهم بالنار\nراح جاب مسدس فعلا وقتل ال ٨ وسلم نفسه للشرطة ... https://t.co/DOCXZ6NowV,من ٥ ايام حصلت جريمة في صربيا طفل عمره ١٤ سنة كان بيتعرض للتنمر من ٧ بنات و ولد زمايله في المدرسة وقعد يهددهم بالقتل زمايله دول زادوا من التنمر وبقوا يعملوا صوت كده بيو بيو تريقة يعني انو هيضربهم بالنار راح جاب مسدس فعلا وقتل ال ٨ وسلم نفسه للشرطة ...


In [18]:
df['text_arabic'] = df['text_arabic'].fillna('')

In [19]:
df['text_length'] = df['text_arabic'].apply(lambda x: len(x))

## Tweet Emojis

In [20]:
# Define emoji regex pattern
def extract_emojis(text):
    try:
        return demoji.findall(text)
    except:
        return ''

In [21]:
test = 'رسميا : الماجيكو محمد ابو تريكه رئيسا لنادي مانشيستر يونايتد بعد شراء النادى من قبل رجل الأعمال القطري ماجد الخليفي .. اللي فرحان للماجيكو يوريني نفسه كدا 😘😘😘❤ https://t.co/uA0skv7B3h	'
emojis = extract_emojis(test)
emojis

{'😘': 'face blowing a kiss', '❤': 'red heart'}

In [22]:
# Create a new column 'emojis' with extracted emojis
df['emojis'] = df['text'].apply(extract_emojis)

In [23]:
df[['text', 'emojis']].sample(10)

,text,emojis
14305,👆🔴❗شرطي سويدي يخنق طفلاً مسلماً يردد الشهادتين ❗\n🔴هذا طبعاً ليس إرهاباً طالما إن الضحيه مسلم ❗ http://t.co/zlmIBE3rLC,"{'👆': 'backhand index pointing up', '❗': 'red exclamation mark', '🔴': 'red circle'}"
19394,"ما تقول إلا قناة العربية 🤣🤣🤣\nمترجم قناة العربية أثناء كلمة الملك تشارلز الثالث: ""أنا في غاية السعادة بوفاة أمي الملكة إليزابيث""*😃😃😃😃😃 https://t.co/AELZebtf0c","{'😃': 'grinning face with big eyes', '🤣': 'rolling on the floor laughing'}"
15835,الدفاع المدني⁩ يسيطر على حريق بأحد المباني قيد الانشاء في منطقة #لوسيل دون مصابين https://t.co/9IAXXrVEZy,{}
13442,بركان سيميرو في إندونيسيا يقذف حممه لعنان السماء (شاهد)\n\nبركان سيميرو يبلغ ارتفاعه 3676 مترًا ، يقع في الإقليم الشرقي لإندونيسيا ، \n\n - صدر إنذار في البلاد ، ووضع آلاف الأشخاص في ملاجئ مؤقتة.🇮🇩\n\nاللهم سلم https://t.co/2XJl9E0XvL,{'🇮🇩': 'flag: Indonesia'}
11041,إعصار قمعي ضخم محمل بالأتربة والغبار يضرب الطائف ويسبب بعض التلفيات.\n\nالحمد لله على سلامتكم يا أهل الطايف \n#الطايف_الان https://t.co/wGp6wAOlMO,{}
19523,و كانـت تَـشـتَـهِـر #الملكة_إليزابيث الـرّاحلة\nبالكرم و فِعـل الخير ، فكانت تَٰـنـهَـبُ ثَـرَواتِ\nالشعوب ثُمّ تُـلـقـي عَليهِم الفُـتـاتَ كالبَهائِم !\n\nأتعبتِ الملوك من بعدِك يا #إليزابيث_الثانية https://t.co/U57a0fVYwC,{}
19320,لوحة ابو جعفر المنصور تزين أحد جدران قصر باكنغهام https://t.co/VRl6lqB05b,{}
7115,السيسي يوجه بصرف 3 مليون دولار لأسر الجنود الإسرائيليين الذي قتلهم الجندي_مصري.. محمد صلاح.. \nهذا احلا بزي في الدنيا... وكمان قام بسجن كل أفراد وأقارب #الشهيد_محمد_صلاح .. وامر بمصادرة جميع ممتلكات عائلته.. \nهكذا اصبح واقع المنافق السيسي .\nhttps://t.co/XHYzNKDzPM https://t.co/8KhhTGFMdq,{}
16184,شاهد .. رد فعل #ترامب لحظة سماعه خبر وفاة ملكة #بريطانيا https://t.co/5wyzgr3ZFK,{}
17771,الحجاج عام 1934م يأتون من المغرب ليقابلوا حجاج الجزائر .. ثم تونس .. ثم ليبيا .. ليقابلوا الحجاج المصريين ويسيروا سوياً إلى مكه المكرمة.\n\nصوره التقطها مصور فرنسى من الجو لحُجاج بيت الله وهم بالقرب من قرية الجحفة بالحجاز ، وقبيل تفعيل حدود سايكس بيكو.\n\nفلا جوزات ولا تأشيرات مرور. https://t.co/x0nmIVnvKA,{}


In [24]:
def count_emojis(text, emojis):
    emoji_count = {}
    if len(emojis) > 0:
        for emoji in emojis.keys():
            emoji_count[emoji] = text.count(emoji)
    return emoji_count

In [25]:
df['emojis_count'] = df.apply(lambda row: count_emojis(row['text'], row['emojis']), axis=1)

In [26]:
df[['text', 'emojis', 'emojis_count']].sample(5)

,text,emojis,emojis_count
6661,الجندي المبتسم بالسيلفي ده اسمه محمد المعتز رشاد واستشهد 2016 في رفح\nاما الشهيد البطل فجر اليوم على حدود مصر فلسطين قتل 3 مجندين صهاينة واصاب 2 ، دي صورته الباهتة \nبلاش نخلط الأمور عشان كل واحد فيهم له قصة بطولية مختلفة وربنا يرحم شهداء المسلمين \n#جندي_مصري https://t.co/aguYAR6pxT,{},{}
969,حمدي جوارا: ما يتداول عن إسلام اللاعب الدولي الايفواري السابق دروغبا ليس صحيحا، فقد أكد أحد الزملاء أنه زار قريته بالأمس واستقبل جماعة من المسلمين ودعوا له بخير وأمّن رافعا يديه معهم للدعاء ..وهذا لا يعني أنه أسلم .. \n\nنسأل الله أن يهديه إلى الإسلام \n\n#دروغبا #دروجبا #Drogba https://t.co/KHMc83vxjP,{},{}
7795,من مصر الي السودان. \nدورنا الحضاري والتاريخي تجاه الانسانية وبغض النظر عن مواقفهم المخزيه تجاهنا يجعلنا دائما نتصرف تصرف الكبير\nوفي زمن لم تستطيع فيه الدول الغربية من اجلاء رعاياها ناهيك عن إرسال مساعدات. https://t.co/DYNgixXFS2,{},{}
16140,@Rich_in_Rahman كتب الله سبحانه الخير لمورغان فريمان في آخر عمره،غير اسمه الى بلال ويرغب أن يعمل مؤذن بمساجد دولة قطر،بمثل هذه المشاهد العظيمه نفتخر بقادة الدول اللذين همهم هو الاسلام وثقافة العرب العريقة بعكس غيرهم همهم تغريب الشعوب والعياذ بالله.عفيه عفيه عليك شاعر راهي وتستاهل دولة قطر 👍 https://t.co/6q909VzBqo,{'👍': 'thumbs up'},{'👍': 1}
12874,"الداعية #حسن_الحسيني ""يتبرأ"" من فيديو تداوله ذباب النظام المغربي حول الجزائر https://t.co/qxI2VBMBUr",{},{}


In [27]:
df['emojis_total_count'] = df['emojis_count'].apply(lambda x: sum(x.values()))

In [28]:
df[['text', 'emojis', 'emojis_count', 'emojis_total_count']].sample(5)

,text,emojis,emojis_count,emojis_total_count
1655,بيدخلونه السجن \n\nقلب الطاولة عليهم ‼️ https://t.co/jADkrt1mRM,{'‼️': 'double exclamation mark'},{'‼️': 1},1
6716,*الان .. بدأت مراسم احياء الاربعين في نهاية شارع فلسطين ( حسينية الامام الخميني ) بحضور السيد القائد، الامام السيد علي الخامنئي الحسيني، وعدد من الهيئات الطلابية للجامعات الايرانية.*\n\n🔹*المصدر وكالة تسنيم للانباء*\n@TasnimNews https://t.co/ehRINTEA4Y,{'🔹': 'small blue diamond'},{'🔹': 1},1
8354,بالفيديو | رونالدو يوجه رسالة إلى أطفال سوريا: «أنا لاعب مشهور لكن أنتم الأبطال الحقيقيون.. لا تفقدوا الأمل»\nhttps://t.co/xwHapMRow8 https://t.co/6WCt9Taqc3,{},{},0
20006,🚨وفاة السفير السعودي محمد القحطاني اثناء كلمة له ألقاها في القاهرة خلال مؤتمر ، وكانت اخر كلمة له وهو يمدح السيسي قائلا عنه بانه عميد الانسانية و رجل السلام https://t.co/gDIhLRTt4L,{'🚨': 'police car light'},{'🚨': 1},1
15423,🚨🚨🚨🚨بالاك وتصريح تاريخي: \n\nكيف نريد الفوز بكأس العالم للرجال ونحن منذ أن بدأت البطولة ندعم قضيه لا تمت للرجال بصله!♥️ https://t.co/bHcHCJj9cA,"{'♥️': 'heart suit', '🚨': 'police car light'}","{'♥️': 1, '🚨': 4}",5


## tweets annotations

In [29]:
df[['text', 'annotations_text', 'annotations_type']].sample(10)

,text,annotations_text,annotations_type
14997,@SakerSport عاجل🚨🚨🚨\nوفاة اسطورة البرازيل بيليه \nوداعا 🇧🇷🇧🇷,[],[]
19808,اللهم لطفك يااااا رب\nالآن الآن علي CNN محطات الأرصاد الأمريكية تعلن أن السودان سوف يتعرض لأمطار لم تحدث منذ 100 عام https://t.co/idYC5v7wti,['السودان'],['Place']
5873,فيضانات تركيا هذا هو نفس المكان الذي ضربه الزلزال المدمر \nhttps://t.co/SekCkCZH0L,['تركيا'],['Place']
10337,المغربي عبدالحق نوري صحي من غيبوبة مدتها سنتين و٩ أشهر تقريباً الحمد لله\n\nخبر أسعدني كثيرا والله https://t.co/gdnDO0LJxF,[],[]
7425,مشادة كلامية بين ممثل قطر وسفير السعودية في الجامعة العربية https://t.co/ovgEacs10l https://t.co/ef9zpYvC13,"['قطر', 'السعودية']","['Place', 'Place']"
6429,@mhalshaikh @InterMiamiCF جانب من حفل إستقبال ميسي رجعوا البشوت https://t.co/3ydovwIuIi,['ميسي'],['Person']
2480,والدة النجم البرازيلي #نيمار ترتدي النقاب وتمتطي الجمل https://t.co/9CnCbHIwNW,[],[]
10037,الان\nيااالله ما اعظمه من مشهد #الاقصى_يستغيث وصواريخ المقاومة تلبي ..\nمن اليمن نقبل أقدام أبطال المقاومة الفلسطينية https://t.co/laDtRjI1aC,['اليمن'],['Place']
791,لليوم الخامس والثمانين.. بركان لا بالما مستمر بإطلاق الحمم \n#شاهد_سكاي https://t.co/ci3kcOVL0s,[],[]
572,طائرات shahed-136 تهاجم أهدافًا في أوديسا فجر اليوم https://t.co/SVKBh7E6Sh,[],[]


#### Convert data type from plain string into list

In [30]:
df['annotations_text'] = df['annotations_text'].apply(ast.literal_eval)
df['annotations_text'].sample(5)

11603    [محمد ابو تريكه, مانشيستر يونايتد, النادى, ماجد الخليفي]
8979                                                           []
11936                                              [لبنان, سورية]
16826                                                          []
2057                                                           []
Name: annotations_text, dtype: object

In [31]:
df['annotations_type'] = df['annotations_type'].apply(ast.literal_eval)
df['annotations_type'].sample(5)

4147                         []
11643                        []
18383                        []
9539                         []
3243     [Place, Place, Person]
Name: annotations_type, dtype: object

#### Calculate annotations count

In [32]:
df['annotations_count'] = df['annotations_type'].apply(lambda x: len(x))

In [33]:
df[['text', 'annotations_text', 'annotations_type', 'annotations_count']].sample(10)

,text,annotations_text,annotations_type,annotations_count
12374,فيديو: عسكري يضرب “حاج” كبير بالسن لأنه حاول ازالة الحاجز !؟ http://t.co/DqljgLm4WH,[],[],0
3105,ماكرون: سحب القوات الفرنسية من النيجر سيكون بطلب من بازوم\n\nأكد الرئيس الفرنسي إيمانويل ماكرون الأحد، أن أي إعادة انتشار محتملة للقوات الفرنسية في النيجر لن تتم سوى بطلب من الرئيس المخلوع محمد بازوم. https://t.co/t8VMtSZJJL,"[إيمانويل ماكرون, محمد بازوم]","[Person, Person]",2
18334,💥💥💥عاااجل🔞🔞\n\nسفير الايراني في لندن في قبضة المتظاهرين..\n\nجيد جدا مادام شرطة بريطانيا تركتهم يدخلون السفارة ويضربون السفير الايراني يعني ان بريطانيا ستغير سياستها في إيران 👇🏿\n\n #كابتشينو_يخطب_هند_القحطاني https://t.co/fLMHgyPcP9,"[لندن, بريطانيا, بريطانيا, إيران]","[Place, Place, Place, Place]",4
3265,🇮🇶🇮🇷 - قوات إقليم كردستان تتمركز على حدود إيران.\nايه العسل ده يا عم الحج https://t.co/eitMl2JDe7,[إيران],[Place],1
15097,وفاة الأسطورة البرازيليه بيليه 💔,[بيليه],[Person],1
20043,دمشق عاصمة الامويين التي حكمت سبع بحار يسرح ويمرح بها الفرس اه ياابابكر وعمر https://t.co/pqF8lm37it,[],[],0
13106,السيسي: \nبتكلم كتير عن ربنا لأنه حبيبي وغالي عليا، \n\nوكمان لأني اللي شايف وكرمه على مصر فوق خيالكم \n\nوممكن تكونوا شايفين\n\n بس أنا شايف أكتر منكم\n https://t.co/xg7Jub9tPQ https://t.co/tZtOPQp6pM,"[السيسي, مصر]","[Person, Place]",2
3463,تركيا 🇹🇷 \n🔞🔞🔞🔞\nإعتداء حوالي 50 شخص تركي على شاب يمني شجاع عمره 15 سنة في #إسطنبول بعد خلاف مع شاب تركي فى مجمع سكنى مشهور⁉️\n\n#اليمن #تركيا #İstanbul https://t.co/88Uhj6nYhk,[تركيا],[Place],1
19796,🔴 عاجل\n\nقبل قليل علي CNN محطات الأرصاد الأمريكية تعلن أن السودان سوف يتعرض لأمطار لم تحدث منذ 100 عام\nاللهم لطفك يااااا رب https://t.co/zFntSOfJg6,[السودان],[Place],1
19955,منقول من حسابات المختلط : مصطفى فتحي مرضاش يجدد للزمالك وقال لضاضا انا كدا كدا هسيبكو واروح التعاون السعودي فبيعوني واستفادوا مني 2 مليون دولار ، راح مرتضي باعه لنادي التعاون السعودي و أول ما قبض ال2 مليون دولار تمنه و راح مبلغ عنه التجنيد واتقبض عليه في المطار وهو مسافر 🤔🤔🤔. https://t.co/ghi0r7Bvf3,"[مصطفى فتحي, للزمالك, التعاون السعودي, التعاون السعودي]","[Person, Organization, Organization, Organization]",4


#### Extract organizations

In [34]:
df['organizations'] = df.apply(lambda x: [], axis=1)
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    items = []
    for i in range(len(row['annotations_type'])):
        if row['annotations_type'][i] == 'Organization':
            items.append(row['annotations_text'][i])
    df.at[index, 'organizations'] = items

In [35]:
df['organizations_count'] = df['annotations_type'].apply(lambda x: x.count('Organization'))

In [36]:
df[['text', 'annotations_text', 'annotations_type', 'organizations', 'organizations_count']].sample(5)

,text,annotations_text,annotations_type,organizations,organizations_count
4980,عاجل.......انفجار غامض ينسف مصفاة النفط باسم تندكويان في مدينة عبادان العربيه في الاحواز المحتله . https://t.co/DQgQmm4blO,"[عبادان, العربيه]","[Place, Place]",[],0
18966,عاجل:\n\nالقوات المسلحه الجنوبية تلقي القبض على #أمير_تنظيم_داعش_الإرهابي باليمن، الملقب بـ (أبو أسامة المهاجر) والمسؤول المالي للتنظيم وعدد من أعضاء التنظيم المرافقين له. https://t.co/0tbx1tKeRJ,[أبو أسامة المهاجر],[Person],[],0
8238,ضبط مخدرات داخل مستودع في خراج فتقا كسروان\nhttps://t.co/k3b2PR5rMw,[],[],[],0
10153,استضافة أشهر لعـ*ــاني الصحـ*ــابة\nعلى شاشات إعلام آل سعود\nلعل خامنئي يرضى\n\nفي منهاج حكم آل سعود .. يبدو أن تعريف العقيدة الصحيحة يبدأ بـــ ((تقدير كلمات لـ*ــعن صحـ*ــابة النبـ*ــي واجلالها))\n\n#ايران #السعودية #العراق https://t.co/nTsJwcNOPZ,[آل سعود],[Person],[],0
17706,صورة لقوافل الحجاج فى الصحراء إلتقطت عام 1934 عندما كان الحجاج يأتون من المغرب ليقابلوا حجاج الجزائر ثم تونس ثم ليبيا ويتقابلون فى النهاية مع الحجاج المصريين وذلك هرباً من عصابات آل سعود ويسيروا سوياً فى قافلة كبيرة إلى مكه المكرمة.\n\n! https://t.co/0nWNXphb12,"[الجزائر, تونس, ليبيا, مكه المكرمة]","[Place, Place, Place, Place]",[],0


#### Extract person

In [37]:
df['persons'] = df.apply(lambda x: [], axis=1)
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    items = []
    for i in range(len(row['annotations_type'])):
        if row['annotations_type'][i] == 'Person':
            items.append(row['annotations_text'][i])
    df.at[index, 'persons'] = items

In [38]:
df['persons_count'] = df['annotations_type'].apply(lambda x: x.count('Person'))

In [39]:
df[['text', 'annotations_text', 'annotations_type', 'persons', 'persons_count']].sample(5)

,text,annotations_text,annotations_type,persons,persons_count
18972,قيادة القوات المشتركة للتحالف :\n\nالقوات المسلحه الجنوبية تلقي القبض على #أمير_تنظيم_داعش_الإرهابي باليمن، الملقب بـ (أبو أسامة المهاجر) والمسؤول المالي للتنظيم وعدد من أعضاء التنظيم المرافقين له.\n\n#67_سنه_بدون_اسيا \n#اليوم_الوطني_السعودي_92 \n#المهره_جنوبيه_طارده_للارهاب \n#الرياض https://t.co/cdMyEgNohB,[أبو أسامة المهاجر],[Person],[أبو أسامة المهاجر],1
2917,تفاصيل هامة تنشر لأول مرة بشأن وفد #الحوثيين الذي وصل #السعودية\nhttps://t.co/mdYKKzEsab\nلمزيد من الاخبار اشتركو بقناتنا على تلجرام\nhttps://t.co/Y0iHOkO8H0 https://t.co/tb2tGblCje,[],[],[],0
12840,مجموعة مسلحة من #الحوثيين دخلوا سوق مازدا في الحصبة #صنعاء ليبتزوا الباعة المتجولين وأصحاب المحلات، ففاض كيل المواطنين وأوسعوهم ضربًا بالعصي 😁. https://t.co/98aJO0rfob https://t.co/Cgyd6mDYid,[],[],[],0
16473,الكفاح نيوز \n\nمنصور نظام الدين .. المدينة المنورة \n\nبمناسبة مونديال كأس العالم – قطر تعطل الدوام الرسمي لمدة شهر تزامنًا مع المونديال https://t.co/atkjJ0cc0n. #المدينة__المنورة,"[المدينة المنورة, قطر]","[Place, Place]",[],0
19674,اليابان توافق على إنجاز مشروع خط TGV خلال سنة 🇯🇵🇹🇳 ونحو إرساء وكالة فضاء يابانية JAXA في تونس 🇯🇵🇹🇳 https://t.co/S4QXJOaIXM,"[اليابان, تونس]","[Place, Place]",[],0


#### Extract places

In [40]:
df['places'] = df.apply(lambda x: [], axis=1)
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    items = []
    for i in range(len(row['annotations_type'])):
        if row['annotations_type'][i] == 'Place':
            items.append(row['annotations_text'][i])
    df.at[index, 'places'] = items

In [41]:
df['places_count'] = df['annotations_type'].apply(lambda x: x.count('Place'))

In [42]:
df[['text', 'annotations_text', 'annotations_type', 'places', 'places_count']].sample(5)

,text,annotations_text,annotations_type,places,places_count
4509,"بعد أيام من إحباط السعودية محاولة تهريب كبتاغون في قلب شحنة رمُان من لبنان.. الكويت تحبط تهريب 16 كيلو مخدرات محشوة في شحنة أسماك ""هامور"" قادمة من إيران\n https://t.co/sI1KNyIczF","[السعودية, رمُان, لبنان, الكويت, إيران]","[Place, Place, Place, Place, Place]","[السعودية, رمُان, لبنان, الكويت, إيران]",5
13404,🚨🚨🚨عاجل : رغم اصابته.. نجم توتنهام هيونج مين سون يعلن مشاركتة فى كأس العالم 🇰🇷🔥🔥 https://t.co/SeggIkPOua,"[توتنهام, هيونج, مين سون]","[Organization, Person, Person]",[],0
9826,"#عاجل | \n\nقيادة القوات المشتركة للتحالف ""تحالف دعم الشرعية في اليمن"":\n\n▪ القوات الخاصة السعودية تلقي القبض على أمير تنظيم داعش الإرهابي باليمن، الملقب بـ ( أبو أسامة المهاجر ) والمسؤول المالي للتنظيم وعدد من أعضاء التنظيم المرافقين له. https://t.co/u6zVBIQt3L","[اليمن, السعودية, أبو أسامة المهاجر]","[Place, Place, Person]","[اليمن, السعودية]",2
16849,“سون هيونج مين” على رأس قائمة كوريا الجنوبية في كأس العالم https://t.co/5vb80E7Mh6,"[سون هيونج, كوريا الجنوبية]","[Person, Place]",[كوريا الجنوبية],1
11512,"🗣️ كريستيانو رونالدو :\n\n"" أنا اشعر بالخيانة من مانشستر يونايتد ، النادي استخدمني كـ عذر لكل الاخفاقات التي حدثت ، لقد أجبروني على الخروج ، المدرب واشخاص آخرين أيضاً ، الأمر ليس جديدًا هناك أشخاص لم يرغبوا بوجودي في النادي مُنذ العام الماضي وليس هذا العام فقط ""","[كريستيانو رونالدو, مانشستر يونايتد]","[Person, Organization]",[],0


#### Extract products

In [43]:
df['products'] = df.apply(lambda x: [], axis=1)
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    items = []
    for i in range(len(row['annotations_type'])):
        if row['annotations_type'][i] == 'Product':
            items.append(row['annotations_text'][i])
    df.at[index, 'products'] = items

In [44]:
df['products_count'] = df['annotations_type'].apply(lambda x: x.count('Product'))

In [45]:
df[['text', 'annotations_text', 'annotations_type', 'products', 'products_count']].sample(5)

,text,annotations_text,annotations_type,products,products_count
4128,رحلة الهروب من المسعودية 😂 https://t.co/FFyfg2epmL,[],[],[],0
16095,@SufianSamarrai مسيرات في شوارع الفلوجة بالأنبار بعد الفوز https://t.co/aJl0okyk19,[],[],[],0
7171,مشاهد من المناورة اليوم\n#قسماً_قادرون_سنعبر \n#مناورة_المقاومة_الإسلامية \n#حزب_الله https://t.co/uF3ZZnSlF3,[],[],[],0
4998,"صاروخ ""تاو"" يبيد مجموعة كاملة من قوات #الأسد في #حماة (شاهد)\nرابط بديل : https://t.co/7vZd5AGowm\nhttps://t.co/ex30yuLXTZ",[],[],[],0
10004,ربع مليون مصلى النهاردة فى رحاب المسجد الاقصى يا ولاد الأحبة عادى https://t.co/EDXYAJcisz,[],[],[],0


#### Hashtags

In [46]:
df['hashtags'].sample(5)

193                          []
18463                        []
4862                         []
8399                         []
421      ['ريف_دمشق', 'جرمانا']
Name: hashtags, dtype: object

In [47]:
# Convert string columns to actual lists
df['hashtags'] = df['hashtags'].apply(ast.literal_eval)

In [48]:
df['hashtags'].sample(5)

18229                                     []
7614     [قمه_جده, ولي_العهد, القمة_العربية]
16248                          [صفعة_ماكرون]
8015                                      []
19007                [اليمن, السعودية, داعش]
Name: hashtags, dtype: object

In [49]:
df['hashtags_count'] = df['hashtags'].apply(lambda x: len(x))

In [50]:
df[['text', 'hashtags', 'hashtags_count']].sample(5)

,text,hashtags,hashtags_count
3846,📸🎦⬇🚨\n\n*إعصار جيبي، أقوى إعصار في اليابان منذ 25 عاما، ضرب المنطقة الغربية من اليابان بعد ظهر اليوم الثلاثاء ليصل إلى اليابسة في الجزء الجنوبي من محافظة توكوشيما، حسبما أفادت هيئة الأرصاد الجوية. وتسبب الإعصار في تعطيل خدمات النقل بشكل كبير.*# https://t.co/oxjQqoGMoc,[],0
8405,"إبراهيموفيتش : "" كل شيء رائع في مونديال قطر ومن يرون عكس ذلك هم الشوا ذ فقط "" 🔥 https://t.co/8TstffcspI",[],0
16009,راية الحشد الشعبي المقدس في ارض البرازيل.\n#الحشد_باقي_وينتشر_على_عناد_ال_سعود https://t.co/ajWZGNB7Ei,[الحشد_باقي_وينتشر_على_عناد_ال_سعود],1
15836,#قطر | السيطرة على حريق بأحد المباني قيد الانشاء في منطقة #لوسيل دون مصابين.\n\n#الخيمة_نيوز #قطر2022 https://t.co/ULVKeQavXw,"[قطر, لوسيل, الخيمة_نيوز, قطر2022]",4
7677,"""#الجمارك"" تحبط تهريب 10 ملايين حبة كبتاجون مخدرة عبر منفذ #البطحاء.\n\n https://t.co/9a9xnNclXJ","[الجمارك, البطحاء]",2


## Tweet context

In [51]:
df['context_domain'] = df['context_domain'].apply(ast.literal_eval)

In [52]:
df['context_entity'] = df['context_entity'].apply(ast.literal_eval)

In [53]:
df[['text', 'context_domain', 'context_entity']].sample(5)

,text,context_domain,context_entity
14339,❗شرطي سويدي يخنق طفلاً مسلماً يردد الشهادتين ❗ً ليس إرهاباً طالما إن الضحيه مسلم ❗ انشروها قبل ما يحذفون المقطع .. http://t.co/OcgRAsuRGd,[],[]
12127,@s_alharbi2020 دخول معدات مصرية 🇪🇬 لإعمار غزة بقرار من رئيس #مصر\nمفيش حد هياخد جنية في ايده\nشركات مصرية هي اللي هتصرف وتعمر يعني لا حماس ولا عباس https://t.co/HOIhfADxHJ,[Unified Twitter Taxonomy],[Popular Videos]
18968,مجلة التايم الأمريكية تختار الرئيس القائد عيدروس الزبيدي كشخصية للعام وهو أول شخصية عربية تتصدر غلاف المجلة. https://t.co/ArQaNTpraQ,[],[]
11408,#أحمد_شوبير يأكل ” عيش وحلاوة ” على الهوا ساخراً من #محمود_طاهر ومجلس إدارة #الأهلي | #مصر_الإخبارية... https://t.co/7MqdNWa8Wd,[],[]
942,وقع قبل لحظات من بدء صلاة الجمعة.. هجوم انتحاري على مسجد شيعي بمدينة بيشاور في #باكستان تسبّـب في مقتل 56 شخصاً على الأقل وإصابة نحو 200 بجروح وتنظيم الدولة يعلن مسؤوليته عنه\n#ألبوم https://t.co/w3O3Xdcqxa,"[Brand Vertical, Entities [Entity Service], Brand, News Vertical, Unified Twitter Taxonomy]","[Entertainment, TV/Movies Related - Entertainment, Al Jazeera, World news, World news]"


In [54]:
df.columns

Index(['tweet_id', 'possibly_sensitive', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'bookmark_count', 'impression_count',
       'user_id', 'lang', 'annotations_text', 'annotations_type', 'hashtags',
       'mentions', 'urls', 'urls_titles', 'urls_desc', 'context_domain',
       'context_domain_desc', 'context_entity', 'context_entity_desc',
       'copyright', 'withheld_country_codes', 'created_at', 'edits_remaining',
       'is_edit_eligible', 'text', 'reply_settings', 'label',
       'user_created_at', 'user_description', 'user_description_urls',
       'user_description_hashtags', 'user_description_mentions', 'location',
       'protected', 'followers_count', 'following_count', 'tweet_count',
       'listed_count', 'user_like_count', 'verified', 'claim_id',
       'text_arabic', 'text_length', 'emojis', 'emojis_count',
       'emojis_total_count', 'annotations_count', 'organizations',
       'organizations_count', 'persons', 'persons_count', 'places',
     

In [55]:
textual_features = ['possibly_sensitive', 'annotations_text', 'annotations_type', 'hashtags',
       'context_domain', 'context_domain_desc', 'context_entity', 'context_entity_desc',
       'edits_remaining', 'is_edit_eligible', 'text', 'reply_settings', 'label',
       'text_arabic', 'text_length', 'emojis', 'emojis_count', 'emojis_total_count', 'annotations_count',
       'organizations', 'organizations_count', 'persons', 'persons_count',
       'places', 'places_count', 'products', 'products_count', 'hashtags_count']

In [56]:
df_textual = df[textual_features]

In [57]:
df_textual.to_excel('data/05-VERA-ARB_Textual.xlsx', index=False)

# 2- User features

#### User mentions

In [58]:
df['mentions'].sample(5)

1511     []
11885    []
18656    []
9156     []
2995     []
Name: mentions, dtype: object

In [59]:
# Convert string columns to actual lists
df['mentions'] = df['mentions'].apply(ast.literal_eval)

In [60]:
df['mentions'].sample(5)

3319    []
7873    []
3942    []
6926    []
661     []
Name: mentions, dtype: object

In [61]:
df['mentions_count'] = df['mentions'].apply(lambda x: len(x))

In [62]:
df[['mentions', 'mentions_count']].sample(5)

,mentions,mentions_count
11165,[],0
19364,[],0
16459,[],0
19751,[],0
2396,[],0


#### User description

In [63]:
df[['user_description', 'user_description_urls', 'user_description_hashtags', 'user_description_mentions']].sample(10)

,user_description,user_description_urls,user_description_hashtags,user_description_mentions
16837,NaN,[],[],[]
9337,رعاة الإبل🐪 وساكني الخيمة⛺ ولنا الفخر\nلنا الافعال ولكم ردح العوالم🎷,['http://noaf1358.sarhne.com'],[],[]
2920,صحفي يمني ـ امارس عملي الصحفي والاعلامي منذ عام 2008 اجيد التحليلات السياسية وكتابة المقالات مديرآ سابقآ لمركز هدى القران للدراسات الاستراتيجية,[],[],[]
14744,مدرس أحياء 🧬🔬\nIn love with #Al_Ahly 🦅 #AC_Milan 🔴⚫\n\nhttps://t.co/7fIDLAOWNL,['https://www.facebook.com'],"['Al_Ahly', 'AC_Milan']",[]
7793,أجمل منصة ترفيهية لأخبار و احداث النجوم والمشاهير حول العالم | الخاص للدعاية و الإعلان,['http://t.me/egphpp'],['فولو'],[]
7968,ناصر الحق ولو بكلمة,[],['فلسطين'],[]
19886,نيوز كوم - كل ما هو حصري . الإعلانات التواصل عن طريق الخاص - مرخصة من قبل وزارة الإعلام,[],[],[]
15073,اللهم صلي وسلم على نبينا محمد 🌹,[],['قنوات_الكاس'],[]
3518,صحيفة تصدر عن مؤسسة المرصد للنشر الإلكتروني | للتواصل info@al-marsd.com,[],[],[]
5845,( البحث عن موقع في عالم البزنزس) التفائل سمتي,[],['السعودية'],[]


In [64]:
df['user_description_text'] = df['user_description'].apply(filter_text)

In [65]:
df[['user_description', 'user_description_text']].sample(5)

,user_description,user_description_text
17037,”إنّ خيرَ اللهِ مُقبِل .. كلّ ما ترجوهُ آتٍ“,إنّ خيرَ اللهِ مُقبِل .. كلّ ما ترجوهُ آتٍ
19847,سينالك الاذى من الذين يعيشون في وطنك كالغرباء؛\n؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛ \nلاتبرر فشلك بتقليل شأن الاخرين ..\nواعلم ان نجاحك مرهون باخلاقك؛؛,سينالك الاذى من الذين يعيشون في وطنك كالغرباء؛ ؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛؛ لاتبرر فشلك بتقليل شأن الاخرين .. واعلم ان نجاحك مرهون باخلاقك؛؛
7569,حساب يلقي الضوء على المجتمع السعودي من الداخل ويرصد الأخبار العاجلة والمهمة، ويتابع التطورات ومدى انعكاسها على المواطن,حساب يلقي الضوء على المجتمع السعودي من الداخل ويرصد الأخبار العاجلة والمهمة، ويتابع التطورات ومدى انعكاسها على المواطن
8387,"(Lazy) Blogger, Tech Lover & PlayStation Player. I tweet my own thoughts about anything, literally .. つづく",. ..
12368,المحترم يفرض علي ان احترمه غيرتي علي الله ورسوله ووطني ومليكي اعتز بحبي لكيان النادي العالمي النصر السعودي,المحترم يفرض علي ان احترمه غيرتي علي الله ورسوله ووطني ومليكي اعتز بحبي لكيان النادي العالمي النصر السعودي


In [66]:
df['user_description_emojis'] = df['user_description'].apply(extract_emojis)

In [67]:
df[['user_description', 'user_description_emojis']].sample(10)

,user_description,user_description_emojis
11324,اخبار عاجلة عالمية,{}
8932,يومية شاملة تصدر عن دار الوطن للطباعة والنشر والتوزيع – تأسست سنة 1995م ..تابعوا كل الأخبار عبر هاشتاج #جريدة_الوطن_القطرية 📰🇶🇦,"{'🇶🇦': 'flag: Qatar', '📰': 'newspaper'}"
15927,إنسان عايش يتأمل .. ويأمل .. مسلم .. عربي .. السعودية أولوية .,{}
5486,NaN,
8722,NaN,
13291,عراقي🇮🇶 صدري,{'🇮🇶': 'flag: Iraq'}
54,"Author/Editor-in-Chief of American and International Affairs/ Dr.SamXPlatform/ Political Activist, Ph.D.Cognitive Psychology/Doctoral Academic Researcher at GCU",{}
5133,⛈️🌴,"{'🌴': 'palm tree', '⛈️': 'cloud with lightning and rain'}"
3281,إعلامي حر.. إلهي نفسي وانفاسي وكل شعرةٍ وذرةٍ تقول صوتاً وصمتاً واعجاماً وإعراباً : لاغنى لي طرفة عينٍ عن رحمتك ورضاك وغفرانك,{}
3344,أكتب ما يحلو لي .. ولست مسؤول عن إستيعابك الخاطئ لتغريدتي,{}


In [68]:
df['user_description_emojis_count'] =  df.apply(lambda row: count_emojis(row['user_description'], row['user_description_emojis']), axis=1)

In [69]:
df[['user_description', 'user_description_emojis', 'user_description_emojis_count']].sample(10)

,user_description,user_description_emojis,user_description_emojis_count
13352,‏‏‏‏جريدة رياضيه كويتية يمثِلُها قانونياً المحامي خالد السلطان بنيد القار ق٣ ش٦٤ مجمع ديما الدور الخامس مكتب ١٦ / ٩٩٣٣٣٠٠٩ /٦٧٦٢٦٢٦٩ ترحيص وزارةرقم 391-2020,{},{}
18053,ان جاك مني طيب تستاهل الطيب وان جاك غير الطيب هذي سواياك Real Madrid 👑 Powerlifting 🏋🏼‍♂️ 💛Qadisiyah 🎬 Movie 🍿🎮 Games,"{'🏋🏼‍♂️': 'man lifting weights: medium-light skin tone', '💛': 'yellow heart', '👑': 'crown', '🍿': 'popcorn', '🎮': 'video game', '🎬': 'clapper board'}","{'🏋🏼‍♂️': 1, '💛': 1, '👑': 1, '🍿': 1, '🎮': 1, '🎬': 1}"
13292,السيد محمد الصدر .. بحر كلما شربنا منه، تـُقـنا الى مائهِ وفيضه،وكلما أبحرنا فيه اكتشفناه أكثر وأكثر,{},{}
6458,‏‏‏‏‏‏‏‏‏‏‏‎‎‎‎‎‎‎‎‎‎‎#شبوه_الجنوب_العربي\nناشط حقوقي جنوبي\nمدافع عن حقوق الانسان فوق أي أرض وتحت أي سماء\nHuman Rights Activist,{},{}
11767,تابعتك رد المتابعة,{},{}
12617,NaN,,{}
11387,The official Group of Zamalek ⚪️🔴 Supporters in UAE 🇦🇪,"{'🔴': 'red circle', '🇦🇪': 'flag: United Arab Emirates', '⚪': 'white circle'}","{'🔴': 1, '🇦🇪': 1, '⚪': 1}"
4624,ممنوع السياسة 🚫✋🏻\n🖤 صراحة 🖤\nhttps://t.co/z7Rg1aCP0P,"{'✋🏻': 'raised hand: light skin tone', '🚫': 'prohibited', '🖤': 'black heart'}","{'✋🏻': 1, '🚫': 1, '🖤': 2}"
11964,مدوُن | ككهلٍ مرَّ بالعشرينَ سهوًا,{},{}
9116,يومية - سياسية - مستقلة | مؤسسها: سمو الشيخ سعيد بن سيف آل نهيان | رئيس التحرير: د.عبدالرحمن الشميري #الوطن_لكل_الوطن,{},{}


In [70]:
df['user_description_urls'] = df['user_description_urls'].apply(ast.literal_eval)

In [71]:
df['user_description_urls_count'] = df['user_description_urls'].apply(lambda x: len(x))

In [72]:
df[['user_description', 'user_description_urls', 'user_description_urls_count']].sample(10)

,user_description,user_description_urls,user_description_urls_count
5314,سبحان الله وبحمده سبحان الله العظيم اللهم صل وسلم وبارك على نبينا محمد💚ملكي ابا من ابا وزعل من زعل متعصب وبقوه للاهلي 💚🤍💪 حساب جديد ،،مسوق عقاري بمكه 🌹,[],0
19074,واحد من سرب اولئك الرجال الذين اقسموا علي حفظ امن هذه البلد,[],0
15778,مالذي سوف انشره هنا ..\nلا احد يعلم...\nتابعونا...,[],0
4892,abdelkader moulaï,[],0
19143,💞 ثم يعوضك الله بمن يليق بقلبك 💞.,"[http://facebook.com/thawretshaab11, http://facebook.com/thawretshaab01, http://instagram.com/thawretshaaab/, http://threads.net/@thawretshaaab]",4
3468,حساب شخصي منوع,[https://wtspee.com/966503856594],1
14096,( آغرد بكل خير وكل مايرضي خالقي ),[],0
10698,"مواطن عدني ,جنوبي الهوية إنتقالي الانتماء ™ حسابي الاساسي 👈 @M1_Noman",[],0
6194,NaN,[],0
17759,وما من كاتب إلاّ سيفنى ويُبقي الدهرُ ما كتبت يداه فلا تكتب بخطك غير شيءٍ يسرّك في القيامة أن تراه....,[],0


In [73]:
df['user_description_hashtags'] = df['user_description_hashtags'].apply(ast.literal_eval)

In [74]:
df['user_description_hashtags_count'] = df['user_description_hashtags'].apply(lambda x: len(x))

In [75]:
df[['user_description', 'user_description_hashtags', 'user_description_hashtags_count']].sample(10)

,user_description,user_description_hashtags,user_description_hashtags_count
12030,موقع مهتم بالشأن السوري وتداعياته في المنطقة، يقدم الأخبار والتحليلات والدراسات والتقارير ومواد الرأي، ويوثق أحداث القضية السورية ويهتم بأبرز الأخبار العالمية.,[],0
8478,‏‏‏‏‏‏‏‏‏‏‏انا مصمم يوفنتيني .. ابن مكة المكرمة\nعضو في فريق ‎‎‎‎‎@CurvaJ,[],0
4735,لا اله الا انت سبحانكً أني كنت من الظالمين👌,[],0
13523,https://t.co/e3C9tMzL3Q…,[],0
13282,لاتسأل الطغات لماذا طغو بال أسأل العبيد الماذا ركعو,[],0
5923,Most Popular Sports News Website in Egypt,[],0
5686,نائب، رئيس، نقابه، الصحفيين، الجنوبيين في العاصمه عدن,"[السيسي, تحيا_مصر]",2
2627,"24hrs Lebanese, Arab and International news 🌍 IG: thelbnews",[],0
17058,مهندس مدنى\nاحب ‏‏‏كرة قدم العالميه,[],0
18754,نحن المنتظرون,[],0


In [76]:
df['user_description_mentions'] = df['user_description_mentions'].apply(ast.literal_eval)

In [77]:
df['user_description_mentions_count'] = df['user_description_mentions'].apply(lambda x: len(x))

In [78]:
df[['user_description', 'user_description_mentions', 'user_description_mentions_count']].sample(5)

,user_description,user_description_mentions,user_description_mentions_count
17278,لاتصدق مايقال لك...صدق ماتراه بعينك ....ويشعر به قلبك,[],0
11790,عضو مجلس إدارة نادي النصر السعودي في المستقبل,[],0
18492,‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏ تغيب وتفقد التواصل الذي تبني عليه علاقاتك مع الاخرين هذا ما يجعل نفسك اقرب الناس إليك،، أعاني أحيانا من قلة الذكاء\nوأحيانا من شدته,[],0
16138,الحمد الله,[],0
15163,I find myself happy with the simple things. I appreciate the blessing God gave me\nDirector at Mesopotamia Trading Limited,[],0


In [79]:
df['user_description_mentions_count'].value_counts()

user_description_mentions_count
0    20084
Name: count, dtype: int64

In [80]:
df['user_description'].fillna('', inplace=True)

In [81]:
df[['user_id', 'user_description', 'user_description_text', 'user_created_at']].sample(5)

,user_id,user_description,user_description_text,user_created_at
14203,524076266,,,2012-03-14T06:33:01.000Z
18357,1407093972254834690,Yemen- Marib,,2021-06-21T21:52:22.000Z
6103,717180854989139968,🟢الذياب من القني من شمر / الزعيم الملكي💙 // حسابي/ ديني/ اخباري / ثقافي /فكاهي / رياضي //اتشرف بمتابعة الجميع🌹وطني السعوديه قبلة المسلمين خط احمر🇸🇦,الذياب من القني من شمر الزعيم الملكي حسابي ديني اخباري ثقافي فكاهي رياضي اتشرف بمتابعة الجميعوطني السعوديه قبلة المسلمين خط احمر,2016-04-05T02:43:35.000Z
16227,305186650,حساب إخباري إعلامي بطريقة أخرى,حساب إخباري إعلامي بطريقة أخرى,2011-05-25T19:56:34.000Z
5585,1492122003314257923,صفحة خاصة بمجلة أزهار Beirut-lebanon مجلة ازهار ترصد كل الأخبار على مدار الساعة سياسية اجتماعية – ثقافية - شاملة تصدرمن لبنان وتوزع على كافة الدول العربية,صفحة خاصة بمجلة أزهار مجلة ازهار ترصد كل الأخبار على مدار الساعة سياسية اجتماعية ثقافية شاملة تصدرمن لبنان وتوزع على كافة الدول العربية,2022-02-11T13:03:28.000Z


In [82]:
df.columns

Index(['tweet_id', 'possibly_sensitive', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'bookmark_count', 'impression_count',
       'user_id', 'lang', 'annotations_text', 'annotations_type', 'hashtags',
       'mentions', 'urls', 'urls_titles', 'urls_desc', 'context_domain',
       'context_domain_desc', 'context_entity', 'context_entity_desc',
       'copyright', 'withheld_country_codes', 'created_at', 'edits_remaining',
       'is_edit_eligible', 'text', 'reply_settings', 'label',
       'user_created_at', 'user_description', 'user_description_urls',
       'user_description_hashtags', 'user_description_mentions', 'location',
       'protected', 'followers_count', 'following_count', 'tweet_count',
       'listed_count', 'user_like_count', 'verified', 'claim_id',
       'text_arabic', 'text_length', 'emojis', 'emojis_count',
       'emojis_total_count', 'annotations_count', 'organizations',
       'organizations_count', 'persons', 'persons_count', 'places',
     

In [83]:
df['user_id'] = df['user_id'].astype(str)

In [84]:
user_features = ['mentions', 'user_description', 'user_description_urls',
       'user_description_hashtags', 'user_description_mentions', 'protected', 
       'followers_count', 'following_count', 'tweet_count',
       'listed_count', 'user_like_count', 'verified', 'user_description_text',
       'user_description_emojis', 'user_description_emojis_count',
       'user_description_urls_count', 'user_description_hashtags_count',
       'user_description_mentions_count', 'label']

In [85]:
df_user = df[user_features]

In [86]:
df_user.to_excel('data/05-VERA-ARB_user_features.xlsx', index=False)

# 3- Network features

In [87]:
df['reply_settings'].value_counts()

reply_settings
everyone          19911
mentionedUsers       92
following            81
Name: count, dtype: int64

In [88]:
network_features = ['reply_settings', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'bookmark_count', 'impression_count']

In [89]:
df[network_features].sample(10)

,reply_settings,retweet_count,reply_count,like_count,quote_count,bookmark_count,impression_count
19070,everyone,0,0,0,0,0,0
19182,everyone,0,0,0,0,0,0
10659,everyone,36,22,261,2,2,5776
3436,everyone,0,0,0,0,0,0
6590,everyone,1,2,8,1,0,497
19477,everyone,0,0,0,0,0,0
13289,everyone,5,11,46,2,1,0
4711,everyone,164,23,160,11,0,0
12005,everyone,1,0,5,0,0,0
15900,everyone,0,1,0,0,0,0


In [90]:
df_network = df[network_features]

In [91]:
df_network.to_excel('data/05-VERA-ARB_network.xlsx', index=False)

# 4- Multimedia features

In [92]:
df['urls'].sample(5)

17137                                                                                                                                                                                                                                                                                                      []
17514                                                                                                                                                                                                                              ['https://twitter.com/Emarati_Marshal/status/1586642799906230272/video/1']
435                                                                                                                                                                                                                                  ['https://twitter.com/ahmadhgurashi/status/1641212188474892289/video/1']
4948                                                                                          

In [93]:
# Convert string columns to actual lists
df['urls'] = df['urls'].apply(ast.literal_eval)

In [94]:
df['urls_titles'] = df['urls_titles'].apply(ast.literal_eval)

In [95]:
df['urls_desc'] = df['urls_desc'].apply(ast.literal_eval)

In [96]:
df['urls'].sample(5)

9074     [https://twitter.com/AbdullahAlSafi4/status/1268291021173112838/video/1]
11776         [https://twitter.com/deaf_91111/status/1615082202630422533/video/1]
16865    [https://twitter.com/TheGaff09098065/status/1592832947048312832/photo/1]
14022         [https://twitter.com/Almaskri07/status/1560231506660003846/photo/1]
18531    [https://twitter.com/AhmadSh69868621/status/1575185650206224384/video/1]
Name: urls, dtype: object

In [97]:
df['urls_count'] = df['urls'].apply(lambda x: len(x))

In [98]:
df[['urls', 'urls_count']].sample(5)

,urls,urls_count
4194,[],0
7398,[https://twitter.com/amal_yousef9/status/1464995996769861632/photo/1],1
14584,[https://twitter.com/malashmaliy222/status/1608268616943767552/video/1],1
13163,[],0
14133,[https://twitter.com/AFeras1430/status/768714364669030400/video/1],1


In [99]:
df[['urls', 'urls_titles', 'urls_desc']].sample(10)

,urls,urls_titles,urls_desc
17393,[https://twitter.com/Omar_Madaniah/status/1587135130690273285/photo/1],[],[]
1603,[],[],[]
9718,[https://twitter.com/km_1899/status/1644917249101242368/photo/1],[],[]
9719,[https://twitter.com/ii305v/status/1644917206097051648/photo/1],[],[]
18924,"[http://alsabahpress.com, https://twitter.com/AlsabahMedia/status/1564995243983998980/photo/1]",[],[]
7239,[https://twitter.com/bandershaya/status/1660657250099134471/photo/1],[],[]
14610,[],[],[]
13489,[https://twitter.com/PhobiaMad/status/1609526829320228864/photo/1],[],[]
17496,[https://twitter.com/VIPhamadah74/status/1585577633085100034/photo/1],[],[]
13021,[https://twitter.com/salman19761976/status/1614489114027458570/video/1],[],[]


In [100]:
import glob
import json
json_files = glob.glob("data/extracted_tweets/*.json")

In [101]:
tweets_media_data = []
tweet_attachments = []
tweet_ids = set()  # Create a set to store unique tweet IDs

for file in json_files:
    with open(file, 'r') as f:
        includes_media = []
        json_data = json.load(f)
        
        includes_media = json_data['includes']['media']
        df_media_file = pd.DataFrame(includes_media)
        
        for tweet in json_data['data']:
            tweet_id = tweet['id']
            
            if tweet_id not in tweet_ids:  # Check if tweet ID is already processed                
                tweet_attachments = []
                media_types = []
                media_urls = []
                
                if 'attachments' in tweet:
                    if 'media_keys' in tweet['attachments']:
                        tweet_attachments = tweet['attachments'].get('media_keys', [])
                        
                        for attachment in tweet_attachments:
                            for media_item in includes_media:
                                if media_item['media_key'] == attachment:
                                    #print(media_item)
                                    m_type = media_item['type']
                                    media_types.append(m_type)
                                    try:
                                        media_urls.append(media_item['url'] if m_type == 'photo' else '')
                                    except:
                                        print(media_item)
                                    break
                media_data = {
                    'tweet_id': tweet_id,
                    'media_keys': tweet_attachments, #'' if len(tweet_attachments) == 0 else tweet_attachments,
                    'media_keys_count': len(tweet_attachments),
                    'media_types': media_types,
                    'images_count': media_types.count('photo'),
                    'videos_count': media_types.count('video'),
                    'media_urls': media_urls
                }
                tweets_media_data.append(media_data)
    
    #print('processed', file, '!')
print('Done..............!')

Done..............!


In [102]:
len(tweets_media_data)

20090

In [103]:
df_tweets_media = pd.DataFrame(tweets_media_data)

In [104]:
df_tweets_media.sample(10)

,tweet_id,media_keys,media_keys_count,media_types,images_count,videos_count,media_urls
18840,1571484720893935616,[3_1571484716846485504],1,[photo],1,0,[https://pbs.twimg.com/media/Fc8JBh2XEAA_gZX.jpg]
6909,1644765753806053376,[13_1644765712085295105],1,[video],0,1,[]
1802,1681626633982156801,[7_1681626561143873536],1,[video],0,1,[]
16996,1578753883320758272,[7_1578753832552890368],1,[video],0,1,[]
15802,1586424522663596032,[7_1586424495484665856],1,[video],0,1,[]
17198,591994006810152961,[3_591993979811553280],1,[photo],1,0,[https://pbs.twimg.com/media/CDcvY1vWYAAS_rb.jpg]
19984,1557292910122569728,[3_1557292908356800513],1,[photo],1,0,[https://pbs.twimg.com/media/FZydpjXXkAE4V42.jpg]
10517,663076756866400256,"[3_663076709781086211, 3_663076709982461952]",2,"[photo, photo]",2,0,"[https://pbs.twimg.com/media/CTO4wSFWIAMf9iK.jpg, https://pbs.twimg.com/media/CTO4wS1W4AA7WOC.jpg]"
18656,509598791456935936,[],0,[],0,0,[]
4606,791754890141769728,[],0,[],0,0,[]


In [105]:
df_tweets_media.shape

(20090, 7)

In [106]:
df.shape

(20084, 66)

In [107]:
# Change the type of 'tweet_id' from int to str
df['tweet_id'] = df['tweet_id'].astype(str)

In [108]:
# Change the type of 'tweet_id' from int to str
df_tweets_media['tweet_id'] = df_tweets_media['tweet_id'].astype(str)

In [109]:
df_dataset = pd.merge(df, df_tweets_media, on='tweet_id', how='inner')
df_dataset.shape

(20084, 72)

In [110]:
df_dataset.columns

Index(['tweet_id', 'possibly_sensitive', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'bookmark_count', 'impression_count',
       'user_id', 'lang', 'annotations_text', 'annotations_type', 'hashtags',
       'mentions', 'urls', 'urls_titles', 'urls_desc', 'context_domain',
       'context_domain_desc', 'context_entity', 'context_entity_desc',
       'copyright', 'withheld_country_codes', 'created_at', 'edits_remaining',
       'is_edit_eligible', 'text', 'reply_settings', 'label',
       'user_created_at', 'user_description', 'user_description_urls',
       'user_description_hashtags', 'user_description_mentions', 'location',
       'protected', 'followers_count', 'following_count', 'tweet_count',
       'listed_count', 'user_like_count', 'verified', 'claim_id',
       'text_arabic', 'text_length', 'emojis', 'emojis_count',
       'emojis_total_count', 'annotations_count', 'organizations',
       'organizations_count', 'persons', 'persons_count', 'places',
     

In [111]:
media_features = ['urls', 'urls_titles', 'urls_desc', 'urls_count', 'media_keys', 'media_keys_count',
                  'media_types', 'images_count', 'videos_count', 'media_urls', 'label']

In [112]:
df_media = df_dataset[media_features]

In [113]:
df_media.to_excel('data/05-VERA-ARB_media.xlsx', index=False)

# 5- Spatiotemporal features

In [114]:
df_dataset["location"].fillna("", inplace = True)

In [115]:
df_dataset['location'].sample(10)

16550     Mansoura 🇪🇬 | Jeddah 🇸🇦
19149                            
7345                             
10984                     Bahrain
3847     المملكة العربية السعودية
9459      Khobar،ksa,Old Trafford
13446                            
1756                             
19763                   Abu Dhabi
147                   Middle East
Name: location, dtype: object

In [116]:
df_dataset['location'].value_counts()[:10]

location
                            7400
المملكة العربية السعودية     575
Kuwait                       324
Kingdom of Saudi Arabia      294
Iraq                         268
دولة الكويت                  264
Yemen                        262
Cairo, Egypt                 259
Egypt                        236
Lebanon                      206
Name: count, dtype: int64

In [117]:
spatiotemporal_features = ['location', 'created_at', 'user_created_at', 'label']

In [118]:
df_dataset[spatiotemporal_features].head()

,location,created_at,user_created_at,label
0,مراكش,2023-09-02T12:30:22.000Z,2023-08-06T17:25:40.000Z,1
1,,2023-09-02T11:32:11.000Z,2012-02-14T13:58:41.000Z,1
2,,2023-02-01T16:08:54.000Z,2012-02-14T13:58:41.000Z,1
3,,2022-09-30T23:59:00.000Z,2012-02-14T13:58:41.000Z,1
4,Leo Messi - New Brunswick,2023-09-01T18:11:10.000Z,2022-09-03T14:14:16.000Z,1


## Summary

In [119]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20084 entries, 0 to 20083
Data columns (total 72 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   tweet_id                         20084 non-null  object
 1   possibly_sensitive               20084 non-null  bool  
 2   retweet_count                    20084 non-null  int64 
 3   reply_count                      20084 non-null  int64 
 4   like_count                       20084 non-null  int64 
 5   quote_count                      20084 non-null  int64 
 6   bookmark_count                   20084 non-null  int64 
 7   impression_count                 20084 non-null  int64 
 8   user_id                          20084 non-null  object
 9   lang                             20084 non-null  object
 10  annotations_text                 20084 non-null  object
 11  annotations_type                 20084 non-null  object
 12  hashtags                        

In [120]:
df_dataset['user_id'] = df_dataset['user_id'].astype(str)

In [122]:
df_dataset.to_excel('data/05-VERA-ARB-Cleaned-Dataset.xlsx', index=False)